### Задание

Реализуйте алгоритм классификации метод k ближайших соседей.

Требования к коду:
* Код должен быть хорошо структурирован
* Код должен быть эффективен
* Имплементация должна быть максимально векторизованной и, где это возможно, не использовать циклы

Необходимо реализовать класс KnnBruteClassifier, с реализацией прототипа, представленного ниже.

Должна быть реализована поддержка метрики расстояния L2 (параметр metric) и параметр weights типа 'uniform' и 'distance'.

В качестве входного файла необходимо использовать файл "knn_data_XXX.npy", полученный от бота командой /get seminar04

В качестве решения необходимо отправить боту, указав seminar04 в поле caption,  следующие файлы:
* knn.ipynb - содержит класс, реализующий ваш алгоритм
* results.npy - файл с результатами тестов, который можно будет сгенерировать с помощью этого ноутбука

Для проверки решения после отправки необходимо отправить боту следующую команду:
/check seminar04

В случае возникновения вопросов по интерфейсу смотрите детали реализации класса sklearn.neighbors.KNeighborsClassifier
https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html

In [1]:
import numpy as np

class KnnBruteClassifier(object):
    '''Классификатор реализует взвешенное голосование по ближайшим соседям. 
    Поиск ближайшего соседа осуществляется полным перебором.
    Параметры
    ----------
    n_neighbors : int, optional
        Число ближайших соседей, учитывающихся в голосовании
    weights : str, optional (default = 'uniform')
        веса, используемые в голосовании. Возможные значения:
        - 'uniform' : все веса равны.
        - 'distance' : веса обратно пропорциональны расстоянию до классифицируемого объекта
        -  функция, которая получает на вход массив расстояний и возвращает массив весов
    metric: функция подсчета расстояния (по умолчанию l2).
    '''
    @staticmethod
    def l2(x, y):
        return np.sqrt(np.sum((x - y) ** 2, axis = 1))

    @staticmethod    
    def uniform(distances):
        return np.ones(distances.shape)

    @staticmethod
    def distance(distances):
        return np.ones(distances.shape) / distances
    
    def __init__(self, n_neighbors=1, weights='uniform', metric='l2'):
        
        self.n_neighbors = n_neighbors
        
        if metric == 'l2':
            self.metric_func = self.l2
        else:
            self.metric_func = metric
            
        if weights == 'uniform':
            self.weights_func = self.uniform
        elif weights == 'distance':
            self.weights_func = self.distance
        else:
            self.weights_func = weights
        
     
    def fit(self, x, y):
        '''Обучение модели.
        Парметры
        ----------
        x : двумерным массив признаков размера n_queries x n_features
        y : массив/список правильных меток размера n_queries
        Выход
        -------
        Метод возвращает обученную модель
        '''
        self.x_train = x
        self.y_train = y
        
        return self
    
    def stat_for_y(self, x):
        y_10 = np.zeros((x.shape[0], 10))
        neigh_dist, neigh_indarray = self.kneighbors(x, self.n_neighbors)
        weights = self.weights_func(neigh_dist) 
        
        # проходимся по всем ВЕСАМ и добавляем их куда надо в y_10
        for i in range(weights.shape[0]): 
            for j in range(weights.shape[1]): 
                y_10[i][self.y_train[int(neigh_indarray[i][j])]] += weights[i][j]
                
        return y_10
        
    def predict(self, x):
        """ Предсказание класса для входных объектов
        Параметры
        ----------
        X : двумерным массив признаков размера n_queries x n_features
        Выход
        -------
        y : Массив размера n_queries
        """
        y_10 = self.stat_for_y(x)
                
        return np.argmax(y_10, axis = 1) 
        
    def predict_proba(self, x):
        """Возвращает вероятности классов для входных объектов
        Параметры
        ----------
        X : двумерным массив признаков размера n_queries x n_features
        Выход
        -------
        p : массив размера n_queries x n_classes] c вероятностями принадлежности 
        объекта к каждому классу
        """
        
        y_10 = self.stat_for_y(x)

        return y_10 / np.hstack([np.reshape(np.sum(y_10, axis = 1), (len(y_10),1))] * 10)
        
    def kneighbors(self, x, n_neighbors):
        """Возвращает n_neighbors ближайших соседей для всех входных объектов и расстояния до них
        Параметры
        ----------
        X : двумерным массив признаков размера n_queries x n_features
        Выход
        -------
        neigh_dist массив размера n_queries х n_neighbors
        расстояния до ближайших элементов
        neigh_indarray, массив размера n_queries x n_neighbors
        индексы ближайших элементов
        """
        neigh_dist = np.zeros((x.shape[0], n_neighbors))
        neigh_indarray = np.zeros((x.shape[0], n_neighbors))
        
        for i in range(x.shape[0]):
            distances = self.metric_func(x[i], self.x_train) # посчитали растояние от одного test элемента до каждого элемента в обучающей выборке
            neigh_dist[i] = sorted(distances)[:n_neighbors]
            neigh_indarray[i] = distances.argsort()[:n_neighbors]
            
        return neigh_dist, neigh_indarray

In [2]:
def load_file(filename):
    """
    TODO: Необходимо загрузить файл задания и вернуть словарь с ключами "X_train", "X_test", "y_train"
    """
    data = np.load(filename, allow_pickle = True).tolist()
    
    data['X_train'] = data['X_train'].reshape(len(data['X_train']), -1)
    data['X_test'] = data['X_test'].reshape(len(data['X_test']), -1)
    data['y_train'] = data['y_train'].reshape(len(data['y_train']), -1)
    
    return data 

In [3]:
input_filename = "knn_data_015.npy" #TODO задать путь к входному файлу
data_dict = load_file(input_filename)

In [4]:
model = KnnBruteClassifier(n_neighbors=5, weights='uniform')
model.fit(data_dict["X_train"], data_dict["y_train"])
l2_uniform_n5_y_predict = model.predict(data_dict["X_test"])

In [5]:
model = KnnBruteClassifier(n_neighbors=10, weights='uniform')
model.fit(data_dict["X_train"], data_dict["y_train"])
l2_uniform_10_y_predict = model.predict(data_dict["X_test"])

In [6]:
model = KnnBruteClassifier(n_neighbors=5, weights='distance')
model.fit(data_dict["X_train"], data_dict["y_train"])
l2_distance_n5_y_predict = model.predict(data_dict["X_test"])

In [8]:
output_filename = "results.npy"
result_dict = {
    "input_filename": input_filename,
    "l2_uniform_n5_y_predict": l2_uniform_n5_y_predict,
    "l2_uniform_10_y_predict": l2_uniform_10_y_predict,
    "l2_distance_n5_y_predict": l2_distance_n5_y_predict,
}
np.save(output_filename, result_dict)